# **Fundamentos de Data Science**
Analisis de la Rotación de Clientes en Telecomunicaciones

**Requisitos**<br>
Se debe limpiar y explorar un dataset para los ejecutivos de una empresa de telecomunicaciones que describe tendencias en la rotación de sus clientes.

**Dataset**<br>
Telco Customer Churn - WA_Fn-UseC_-Telco-Customer-Churn.csv <br>
https://www.kaggle.com/datasets/blastchar/telco-customer-churn?resource=download

**Contexto:**<br> 
"Predecir el comportamiento para retener a los clientes. Puede analizar todos los datos relevantes de los clientes y desarrollar programas de retención de clientes específicos". [Conjuntos de datos de muestra de IBM]

**Contenido**<br>
Cada fila representa un cliente, cada columna contiene los atributos del cliente descritos en la columna Metadatos.

El conjunto de datos incluye información sobre:
* Clientes que se fueron durante el último mes: la columna se llama Churn
* Servicios a los que se ha suscrito cada cliente: teléfono, líneas múltiples, Internet, seguridad en línea, copia de seguridad en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas.
* Información de la cuenta del cliente: cuánto tiempo ha sido cliente, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales
* Información demográfica sobre los clientes: género, rango de edad y si tienen parejas y dependientes.

## **Carga y Visualizacion de datos**

* Se importa las librerias necesarias para el analisis del dataset
* Se carga del dataset (archivo .csv) 
* Se visualiza la cantidad d registros y columnas
* Se muestran los primeros 5 registros 

In [1]:
# Importar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import time

In [ ]:
# Cargar el dataset
path = r'C:\Users\romi_\repositorios\Examen_de_Certificacion-Version_C\data\WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)
# Imprime la forma del dataset (registros, columnas)
print(df.shape)
# Muestra los 5 primeros registros del dataset
df.head()

In [ ]:
# Proporciona un resumen del dataframe
df.info()

## **Limpieza de datos con Python**

##### **Detección y eliminación de valores duplicados** 

* Indicación general: Cada registro en el dataset debe ser único.
* El dataset no posee registros duplicados, ninguna accion es necesaria

In [ ]:
# Identificar duplicados
duplicados = df.duplicated()
# Contar el número de duplicados
num_duplicados = duplicados.sum()
print(f"Número de registros duplicados: {num_duplicados}")
df.head()

En caso de necesitar **eliminar duplicados**, el siguiente codigo realizaria la acción

In [ ]:
# Eliminar registros duplicados
df.drop_duplicates(inplace=True)
# Verificar el número de registros después de eliminar duplicados
print(f"Número de registros después de eliminar duplicados: {df.shape[0]}")

# Identificar duplicados
duplicados = df.duplicated()
# Contar el número de duplicados
num_duplicados = duplicados.sum()
print(f"Número de registros duplicados: {num_duplicados}")

##### **Verificación y ajuste de tipos de datos**

* Las columnas deben coincidir con los tipos de datos indicados en el diccionario de datos.

In [ ]:
df.dtypes

In [ ]:
# Iterar a través de cada columna del DataFrame
for columna in df.columns:
    # Obtener los valores únicos de la columna
    valores_unicos = df[columna].unique()
    print(f"Valores únicos en la columna '{columna}': {valores_unicos}")

In [8]:
# Definir el diccionario de tipos de datos
diccionario = {
    'customerID': 'string',
    'gender': 'string',
    'SeniorCitizen': 'int',
    'Partner': 'string',
    'Dependents': 'string',
    'tenure': 'int',
    'PhoneService': 'string',
    'MultipleLines': 'string',
    'InternetService': 'string',
    'OnlineSecurity': 'string',
    'OnlineBackup': 'string',
    'DeviceProtection': 'string',
    'TechSupport': 'string',
    'StreamingTV': 'string',
    'StreamingMovies': 'string',
    'Contract': 'string',
    'PaperlessBilling': 'string',
    'PaymentMethod': 'string',
    'MonthlyCharges': 'float',
    'TotalCharges': 'float',
    'Churn': 'string'
}

In [ ]:
# Función para mapear tipos personalizados a tipos de pandas
def map_tipo(tipo_personalizado):
    mapping = {
        'string': 'string',
        'int': 'int64',
        'float': 'float64'
    }
    return mapping.get(tipo_personalizado, 'object')

# Crear diccionario mapeado para pandas
diccionario_mapeado = {col: map_tipo(tipo) for col, tipo in diccionario.items()}
print("Diccionario Mapeado para pandas:")
diccionario_mapeado

In [10]:
# Función para validar tipos de datos en el DataFrame
def validar_tipos(df, diccionario):
    mismatches = []
    for columna, tipo_esperado in diccionario.items():
        if columna in df.columns:
            tipo_actual = str(df[columna].dtype)
            if tipo_actual != tipo_esperado:
                mismatches.append((columna, tipo_actual, tipo_esperado))
        else:
            mismatches.append((columna, 'No existe en el DataFrame', tipo_esperado))
    return mismatches

In [ ]:
# Ejecutar la validación
mismatches = validar_tipos(df, diccionario_mapeado)

# Mostrar resultados de la validación
if not mismatches:
    print("Todos los tipos de datos son correctos.")
else:
    print("Discrepancias encontradas en los tipos de datos:")
    for columna, actual, esperado in mismatches:
        print(f"- Columna '{columna}': Tipo actual -> {actual}, Tipo esperado -> {esperado}")


La existencia de valores -1 en la columna "TotalCharges" indica que existen valores nulos

In [12]:
def corregir_tipos(df, diccionario):
    for columna, tipo in diccionario.items():
        if columna in df.columns:
            try:
                # Reemplazar cadenas vacías y espacios en blanco por -1
                if tipo == 'float64':
                    df[columna] = df[columna].replace(r'^\s*$', -1, regex=True)
                    
                # Convertir la columna a tipo específico
                df[columna] = df[columna].astype(tipo)
                
                # Manejar -1 de manera adecuada después de la conversión
                if (df[columna] == -1).any():
                    print(f"Advertencia: Existen valores -1 en la columna '{columna}' después de la conversión.")
            except Exception as e:
                print(f"Error al convertir la columna '{columna}' a '{tipo}': {e}")
    return df


In [ ]:
# Corregir los tipos de datos
df = corregir_tipos(df, diccionario_mapeado)

In [ ]:
# Volver a validar
mismatches_post = validar_tipos(df, diccionario_mapeado)

# Mostrar resultados después de la corrección
if not mismatches_post:
    print("Todos los tipos de datos han sido corregidos correctamente.")
else:
    print("Aún existen discrepancias después de la corrección:")
    for columna, actual, esperado in mismatches_post:
        print(f"- Columna '{columna}': Tipo actual -> {actual}, Tipo esperado -> {esperado}")


##### **Consistencia en valores categóricos** 

Identificar y corregir cualquier inconsistencia en los valores categóricos (por ejemplo, ‘yes’, ‘Yes’, ‘YES’).

In [ ]:
# Iterar a través de cada columna del DataFrame
for columna in df.columns:
    # Obtener los valores únicos de la columna
    valores_unicos = df[columna].unique()
    print(f"Valores únicos en la columna '{columna}': {valores_unicos}")

In [16]:
# Definir las columnas categóricas de tu nuevo dataset
categorical_cols = [
    'gender', 
    'SeniorCitizen',  # Esta columna no es categórica, la puedes omitir en la limpieza
    'Partner', 
    'Dependents', 
    'PhoneService', 
    'MultipleLines', 
    'InternetService', 
    'OnlineSecurity', 
    'OnlineBackup', 
    'DeviceProtection', 
    'TechSupport', 
    'StreamingTV', 
    'StreamingMovies', 
    'Contract', 
    'PaperlessBilling', 
    'PaymentMethod', 
    'Churn'
]

In [ ]:
# Diccionario para almacenar los valores únicos antes de la limpieza
valores_unicos = {}
for col in categorical_cols:
    unique_vals = df[col].dropna().unique().tolist()  # Eliminar NaN y convertir a lista
    valores_unicos[col] = unique_vals
    print(f"\nValores únicos en la columna '{col}':")
    print(unique_vals)

# Ruta donde se guardará el archivo JSON con los valores únicos iniciales
ruta_json_inicial = 'C:/Users/romi_/repositorios/Examen_de_Certificacion-Version_C/data/valores_unicos_iniciales.json'
# Guardar el diccionario en un archivo JSON
with open(ruta_json_inicial, 'w', encoding='utf-8') as f:
    json.dump(valores_unicos, f, ensure_ascii=False, indent=4)
    print(f"\nValores únicos guardados en '{ruta_json_inicial}'")


In [18]:
def limpiar_cadena(cadena):
    """
    Limpia una cadena de texto realizando las siguientes operaciones:
    1. Convierte todo el texto a minúsculas.
    2. Elimina caracteres no imprimibles antes de la primera letra y después de la última letra,
       pero mantiene los caracteres internos.
    
    Parámetros:
    - cadena (str): La cadena de texto a limpiar.
    
    Retorna:
    - str: La cadena limpia.
    """
    if isinstance(cadena, str):
        # 1. Convertir todo a minúsculas
        cadena = cadena.lower()
        
        # 2. Eliminar caracteres no imprimibles antes de la primera letra y después de la última letra
        primer_letra = re.search(r'[a-z]', cadena)
        ultima_letra = re.search(r'[a-z](?!.*[a-z])', cadena)
        
        if primer_letra and ultima_letra:
            inicio = primer_letra.start()
            fin = ultima_letra.end()
            cadena = cadena[inicio:fin]
        else:
            cadena = cadena.strip()
        
        return cadena
    return cadena


In [19]:
# Aplicar la limpieza a las columnas categóricas, excluyendo 'SeniorCitizen'
for col in categorical_cols:
    if col != 'SeniorCitizen':
        df[col] = df[col].apply(limpiar_cadena)


In [ ]:
# Verificar los cambios
for col in categorical_cols:
    print(f"\nValores únicos después de limpieza en '{col}':")
    print(df[col].unique())


In [ ]:
# Guardar los valores únicos en un archivo JSON para referencia
valores_unicos_finales = {}
for col in categorical_cols:
    unique_vals = df[col].dropna().unique().tolist()
    valores_unicos_finales[col] = unique_vals

# Guardar en 'valores_unicos_finales.json'
ruta_json_final = 'C:/Users/romi_/repositorios/Examen_de_Certificacion-Version_C/data/valores_unicos_finales.json'
with open(ruta_json_final, 'w', encoding='utf-8') as f:
    json.dump(valores_unicos_finales, f, ensure_ascii=False, indent=4)
    print(f"Valores únicos guardados en '{ruta_json_final}'.")


##### **Manejo de valores faltantes**

* Identifica y maneja cualquier valor faltante en el dataset. 
* Rellena los valores faltantes con un marcador adecuado para el tipo de dato.
* En la columna "TotalCharges", si se encuantran valores iguales a -1 es indicador de un valor de tipo NaN

In [ ]:
# Identificar valores faltantes
qsna = df.shape[0] - df.isnull().sum(axis=0)  # Cantidad de datos sin NAs
qna = df.isnull().sum(axis=0)  # Cantidad de NAs por columna
ppna = round(100 * (df.isnull().sum(axis=0) / df.shape[0]), 2)  # Porcentaje de NAs

# Crear un DataFrame para visualizar los resultados
aux = {
    'datos sin NAs en q': qsna,
    'Na en q': qna,
    'Na en %': ppna
}
na = pd.DataFrame(data=aux)

# Ordenar el DataFrame por el porcentaje de NAs
na_sorted = na.sort_values(by='Na en %', ascending=False)

# Mostrar el DataFrame con el estado de los NAs
print(na_sorted)

* Los registros con TotalCharges = -1 corresponden a valores nulos y representan clientes que no han tenido cargos aplicables. Mantener estos registros en el dataset es fundamental para el análisis de churn, ya que permiten explorar características comunes entre clientes con ausencia de cargos, proporcionando un contexto valioso sobre la retención y abandono de clientes. En lugar de eliminarlos, convertiremos el valor a 0 para facilitar su manejo en análisis futuros, garantizando que no se pierda información relevante<br>
* Se verifica tambien que los registros con valores igual a -1 en la columna "TotalCharges", son los registros que tienen valor igual a 0 en la columna "tenure" y columna "churn" igual a no.

In [ ]:
# Filtrar los registros donde 'TotalCharges' es igual a -1
registros_nulos = df[df['TotalCharges'] == -1]

# Mostrar los registros encontrados
print(registros_nulos)


In [ ]:
# Cambiar los valores -1 a 0 en la columna 'TotalCharges'
df['TotalCharges'] = df['TotalCharges'].replace(-1, 0)

# Verificar si hay valores -1 en la columna 'TotalCharges'
valores_negativos = df['TotalCharges'][df['TotalCharges'] == -1]

if not valores_negativos.empty:
    print(f"Se encontraron {valores_negativos.count()} valores -1 en 'TotalCharges':")
    print(valores_negativos)
    
    # Preguntar al usuario si desea cambiar los valores -1 a 0
    respuesta = input("¿Deseas cambiar los valores -1 a 0 en 'TotalCharges'? (sí/no): ")
    if respuesta.lower() in ['sí', 'si', 'yes']:
        df['TotalCharges'] = df['TotalCharges'].replace(-1, 0)
        print("Los valores -1 han sido cambiados a 0.")
    else:
        print("No se realizaron cambios en la columna 'TotalCharges'.")
else:
    print("No se encontraron valores -1 en 'TotalCharges'.")


In [ ]:
# Verificar si hay valores iguales a 0 en la columna 'MonthlyCharges'
valores_cero = df[df['MonthlyCharges'] == 0]

# Mostrar la cantidad de valores iguales a 0
cantidad_ceros = valores_cero.shape[0]
print(f"Cantidad de valores iguales a 0 en 'MonthlyCharges': {cantidad_ceros}")

# Mostrar los registros que tienen MonthlyCharges = 0
print("Registros con MonthlyCharges = 0:")
print(valores_cero)


In [ ]:
# Reemplazar cadenas vacías en columnas de tipo string por NaN
for col in df.select_dtypes(include='string').columns:
    df[col] = df[col].replace(r'^\s*$', pd.NA, regex=True)

# Verificar el resultado
print(df.isnull().sum())  # Para ver la cantidad de valores nulos por columna


##### **Detección de datos anómalos**

In [ ]:
# Validación 1: TotalCharges menor que MonthlyCharges * tenure
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')  # Convertir a numérico
anomaly_total_charges = df[df['TotalCharges'] < df['MonthlyCharges'] * df['tenure']]

# Validación 2: Todos los servicios son 'No' y Churn es 'No'
anomaly_no_services = df[(df[['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
                                'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                                'StreamingTV', 'StreamingMovies']].eq('No').all(axis=1)) & 
                           (df['Churn'] == 'No')]

# Validación 3: Tenure cero o negativo
anomaly_tenure = df[df['tenure'] <= 0]

# Validación 4: MonthlyCharges menor o igual a cero
anomaly_monthly_charges = df[df['MonthlyCharges'] <= 0]

# Validación 5: Churn es 'Yes' y tenure mayor que cero
anomaly_churn_tenure = df[(df['Churn'] == 'Yes') & (df['tenure'] > 0)]

# Validación 6: Valores faltantes en columnas críticas
missing_values = df.isnull().sum()

# Validación 7: TotalCharges debe ser numérico
non_numeric_total_charges = df[~df['TotalCharges'].astype(str).str.replace('.', '', 1).str.isnumeric()]

# Validación 8: Inconsistencias en el estado del cliente
inconsistent_churn = df[(df['Churn'] == 'No') & 
                         (df[['PhoneService', 'MultipleLines', 'InternetService']].eq('No').all(axis=1))]

# Mostrar conteos de anomalías
print("Cantidad de anomalías:")
print("TotalCharges < MonthlyCharges * tenure:", len(anomaly_total_charges))
print("Todos los servicios son 'No' y Churn es 'No':", len(anomaly_no_services))
print("Tenure cero o negativo:", len(anomaly_tenure))
print("MonthlyCharges <= 0:", len(anomaly_monthly_charges))
print("Churn es 'Yes' y tenure > 0:", len(anomaly_churn_tenure))
print("Valores faltantes en columnas críticas:", missing_values[missing_values > 0].sum())  # Solo muestra los totales de valores faltantes
print("TotalCharges no numérico:", len(non_numeric_total_charges))
print("Inconsistencias en el estado del cliente:", len(inconsistent_churn))


Se procede a dejar la anomalia de "TotalCharges" bajo la consideracion de que podria tratarse de descuentos.<br>
Bajo esa misma premisa, no se procede a ajustar los valores la columna "TotalCharges", imputando "MonthlyCharges" por "tenure", porque se consideran ajustes a la facturacion por metodo de pago o descuento a clientes u otro situacion particulacion como mora

### Exploración de datos con Python:

In [35]:
sns.set(style='whitegrid')

### **Visualizaciones exploratorias univariadas:** 
Crea dos tipos diferentes de visualizaciones univariadas. Cada visualización debe incluir una breve interpretación dentro del archivo de código.

In [ ]:
#Visualización Univariada 1 - Histograma de MonthlyCharges
plt.figure(figsize=(10, 6))
sns.histplot(df['MonthlyCharges'], bins=30, kde=True, color='skyblue')
plt.title('Distribución de Cargos Mensuales (MonthlyCharges)', fontsize=16)
plt.xlabel('Cargos Mensuales', fontsize=14)
plt.ylabel('Frecuencia', fontsize=14)
plt.grid()

# Interpretación
plt.annotate('La mayoría de los clientes tiene cargos mensuales entre 20 y 100.\n'
             'Se observa una ligera inclinación hacia la derecha, sugiriendo que\n'
             'hay algunos clientes con cargos significativamente más altos.',
             xy=(80, 400), xycoords='data', fontsize=12, color='black', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgrey'))

plt.tight_layout()
plt.show()

In [ ]:
#Visualización Univariada 2 - Gráfico de Cajas (Boxplot) de TotalCharges
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['TotalCharges'], color='lightgreen')
plt.title('Boxplot de Cargos Totales (TotalCharges)', fontsize=16)
plt.xlabel('Cargos Totales', fontsize=14)
plt.grid()

# Interpretación
plt.annotate('El boxplot muestra que la mayoría de los cargos totales están\n'
             'concentrados entre 0 y 4000, con algunos valores atípicos.\n'
             'El rango intercuartílico es considerable, indicando variabilidad\n'
             'en los cargos totales de los clientes.', 
             xy=(5000, 0.25), xycoords='data', fontsize=12, color='black', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgrey'))

plt.tight_layout()
plt.show()

In [ ]:
# Bloque 5: Visualización Univariada 3 - Gráfico de Barras de SeniorCitizen
plt.figure(figsize=(10, 6))
sns.countplot(x='SeniorCitizen', data=df, palette='pastel')
plt.title('Cantidad de Clientes por Estado de Senior Citizen', fontsize=16)
plt.xlabel('¿Es Ciudadano Senior?', fontsize=14)
plt.ylabel('Cantidad de Clientes', fontsize=14)

# Interpretación
plt.annotate('La mayoría de los clientes no son ciudadanos senior, lo que\n'
             'sugiere que la base de clientes está compuesta principalmente\n'
             'por personas más jóvenes.', 
             xy=(0.5, 2500), xycoords='data', fontsize=12, color='black', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgrey'))

plt.tight_layout()
plt.show()

### **Visualizaciones exploratorias multivariadas**
Crear dos tipos diferentes de visualizaciones multivariadas. Cada visualización debe incluir una breve interpretación dentro del archivo de código.

In [ ]:
#Visualización Multivariada 1 - Gráfico de Dispersión (Scatter Plot)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='tenure', y='MonthlyCharges', hue='Churn', data=df, palette='viridis', alpha=0.7)
plt.title('Relación entre Tenencia y Cargos Mensuales', fontsize=16)
plt.xlabel('Tenencia (Meses)', fontsize=14)
plt.ylabel('Cargos Mensuales', fontsize=14)

# Interpretación
plt.annotate('Se observa que los clientes con menor tenencia tienden a tener\n'
             'cargos mensuales más altos y son más propensos a cancelar su servicio.\n'
             'A medida que aumenta la tenencia, los cargos mensuales tienden a\n'
             'disminuir, sugiriendo una lealtad creciente entre los clientes.', 
             xy=(10, 80), xycoords='data', fontsize=12, color='black', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgrey'))

plt.tight_layout()
plt.show()



In [ ]:
#Visualización Multivariada 2 - Gráfico de Barras (Bar Plot)
plt.figure(figsize=(10, 6))
sns.countplot(x='Contract', hue='Churn', data=df, palette='pastel')
plt.title('Número de Cancelaciones según Tipo de Contrato', fontsize=16)
plt.xlabel('Tipo de Contrato', fontsize=14)
plt.ylabel('Cantidad de Clientes', fontsize=14)

# Interpretación
plt.annotate('Los clientes con contratos de mes a mes tienen la tasa más alta de\n'
             'cancelación, mientras que los contratos de un año y dos años\n'
             'muestran una mayor retención de clientes.', 
             xy=(0.5, 200), xycoords='data', fontsize=12, color='black', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgrey'))

plt.tight_layout()
plt.show()

### Análisis adicional:
* **Estadísticas descriptivas:** Proporcionar un resumen estadístico del dataset, incluyendo medidas de tendencia central y dispersión para las variables numéricas.
* **Identificación de tendencias:** Analizar y discutir cualquier tendencia notable que observes en los datos, apoyándote en las visualizaciones y estadísticas descriptivas.

In [ ]:
# Identificar columnas numéricas
columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()

print("\nColumnas numéricas identificadas:")
print(columnas_numericas)

def calcular_estadisticas(column, data):
    """
    Calcula estadísticas descriptivas para una columna numérica,
    omitiendo los valores nulos.

    Parámetros:
    - column (str): Nombre de la columna.
    - data (pd.Series): Serie de pandas con los datos de la columna.

    Retorna:
    - dict: Diccionario con las estadísticas calculadas.
    """
    estadisticas = {
        'Cuenta': int(np.sum(~np.isnan(data))),
        'Media': np.nanmean(data),
        'Mediana': np.nanmedian(data),
        'Desviación Estándar': np.nanstd(data, ddof=1),
        'Mínimo': np.nanmin(data),
        'Máximo': np.nanmax(data),
        '25% Percentil': np.nanpercentile(data, 25),
        '75% Percentil': np.nanpercentile(data, 75)
    }
    return estadisticas

# Crear un diccionario para almacenar las estadísticas
estadisticas_dict = {}

# Iterar sobre cada columna numérica y calcular las estadísticas
for columna in columnas_numericas:
    datos_columna = df[columna].values
    estadisticas = calcular_estadisticas(columna, datos_columna)
    estadisticas_dict[columna] = estadisticas

# Convertir el diccionario a un DataFrame para una mejor visualización
estadisticas_df = pd.DataFrame(estadisticas_dict).T  # Transponer para que las columnas sean las filas
estadisticas_df = estadisticas_df[['Cuenta', 'Media', 'Mediana', 'Desviación Estándar', 
                                   'Mínimo', 'Máximo', '25% Percentil', '75% Percentil']]

# Redondear las estadísticas a 2 decimales
estadisticas_df = estadisticas_df.round(2)

# Mostrar el DataFrame formateado
print("\nEstadísticas Descriptivas (Redondeadas a 2 decimales):")
print(estadisticas_df)


##### Variables
* **SeniorCitizen:** Indica si el cliente es un ciudadano de la tercera edad (1) o no (0).
* **tenure:** Representa el número de meses que el cliente ha estado suscrito al servicio.
* **MonthlyCharges:** Es el monto que el cliente paga mensualmente por el servicio.
* **TotalCharges:** Es el cargo total acumulado por el cliente hasta la fecha.

##### Análisis de las Estadísticas Descriptivas:
* **SeniorCitizen:**<br>
La mayoría de los clientes (aproximadamente 84%) no son de la tercera edad.<br>
* **tenure:**<br>
En promedio, los clientes llevan suscritos al servicio alrededor de 32 meses.<br>
La mitad de los clientes lleva menos de 29 meses y la otra mitad más de 29 meses.<br>
Hay una gran variabilidad en la tenencia, con clientes desde los 0 meses hasta los 72 meses.
* **MonthlyCharges:**<br>
El cargo mensual promedio es de $64.76.<br>
La mitad de los clientes pagan menos de $70.35 al mes y la otra mitad más.<br>
El rango de cargos mensuales es bastante amplio, desde $18.25 hasta $118.75.
* **TotalCharges:**<br>
El cargo total promedio es de $2279.73.<br>
Hay una gran dispersión en los cargos totales, con un mínimo de $0 y un máximo de $8684.80.<br>
La mediana es significativamente menor que la media, lo que sugiere que hay algunos clientes con cargos totales muy altos que están elevando el promedio.

In [ ]:
# Identificar columnas numéricas
columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()

# Calcular la matriz de correlación utilizando Pearson
matriz_correlacion = df[columnas_numericas].corr(method='pearson')

# Mostrar la matriz de correlación
print("\nMatriz de Correlación (Pearson):")
print(matriz_correlacion)

# Configurar el tamaño de la figura
plt.figure(figsize=(12, 10))

# Crear el heatmap utilizando seaborn
sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)

# Añadir títulos y etiquetas
plt.title('Matriz de Correlación entre Variables Numéricas', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# Mostrar el gráfico
plt.tight_layout()
plt.show()

##### Interpretacion 
* **SeniorCitizen**
Tiene una correlación positiva muy débil con la antigüedad (0,02). Esto sugiere que ser una persona mayor tiene un impacto insignificante en la duración del servicio.<br>
Tiene una correlación positiva moderada con MonthlyCharges (0,22). Esto indica que los adultos mayores tienden a tener cargos mensuales ligeramente más altos en comparación con los adultos mayores.<br>
Tiene una correlación positiva débil con TotalCharges (0,10). Esto sugiere que los adultos mayores tienden a tener cargos totales ligeramente más altos, probablemente debido a su mayor permanencia en el empleo y a cargos mensuales más elevados.
* **Tenure**
Tiene una fuerte correlación positiva con TotalCharges (0,83). Esto tiene sentido, ya que cuanto más tiempo se quede un cliente, mayores serán sus cargos totales.<br>
Tiene una correlación positiva moderada con MonthlyCharges (0,25). Esto sugiere que los clientes con mayor antigüedad tienden a tener cargos mensuales ligeramente más altos.
* **MonthlyCharges**<br>
Tiene una correlación positiva moderada con TotalCharges (0,65). Esto indica que los clientes con cargos mensuales más altos tienden a tener cargos totales más altos.<br>

**Observaciones generales**
* Seniority y TotalCharges: la correlación más fuerte es entre la antigüedad y los cargos totales, lo cual es esperado.
* SeniorCitizen: Si bien ser una persona mayor tiene cierto impacto en los cargos mensuales y totales, las correlaciones son relativamente débiles.
* MonthlyCharges: Los cargos mensuales tienen un impacto moderado en los cargos totales.

**Consideraciones adicionales**
* La correlación no implica causalidad: aunque la matriz muestra relaciones entre variables, no significa necesariamente que una variable cause la otra.
* Otros factores: Puede haber otros factores que influyan en estas relaciones que no están capturados en esta matriz.


### Entrega
* Un archivo .ipynb claramente comentado con todo el código utilizado para la limpieza y exploración de datos. Para cumplir con los requisitos anteriores, este archivo tendrá un mínimo de 4 visualizaciones.
* Subir un repositorio a una plataforma de control de versiones (por ejemplo, GitHub) con un TAG incluido para verificar la fecha de entrega. El repositorio debe contener el archivo .ipynb.
* **NOTA:** Debes enviar un archivo y un enlace al repositorio con el TAG correspondiente. Las presentaciones de enlaces sin el archivo o sin el TAG serán consideradas no presentadas.

### Notas
* Para evitar que otros estudiantes copien tu trabajo, asegúrate de que el código no esté guardado en un repositorio público.
Para minimizar la deducción de puntos, asegúrate de cargar los requisitos completos.
* Si tienes problemas para cargar archivos en la aplicación de exámenes, por favor NO desbloquees otro examen. Envía el archivo por correo electrónico a tu instructor.
* Puedes usar cualquier recurso que tengas disponible (notas de clase, Google, Stackoverflow, etc.) – PERO DEBES CITAR CUALQUIER FUENTE UTILIZADA. Puedes citar fuentes en forma de comentario de código o una celda de texto con enlaces a cualquier recurso que hayas utilizado. La falta de citas será considerada plagio y resultará en la falla del examen.
* No puedes recibir ayuda de ninguna otra persona en este código. Esto incluye compañeros de clase, exalumnos, tu instructor, etc. La colaboración con otra persona en este examen será considerada plagio y resultará en la falla del examen y puede resultar en la expulsión del programa.
* Se permite el uso de LLM (Copilot, GPT, etc.), pero solo como guía o para depuración. Se espera que los estudiantes pongan su máximo esfuerzo.